In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords

In [ ]:
data = pd.read_csv('dataset/dataset.csv', sep='|')
data.Reviews=data.Reviews.astype(str)

#Transform text to lowercase
data['Reviews'] = data['Reviews'].apply(lambda x: x.lower()) 
#Removing all punctuations and special characters
data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('[,.]', '', x))

In [ ]:
stopwords_list = stopwords.words('english')
data['Reviews'] = data['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [ ]:
print(data)

In [ ]:
conda install -c conda-forge textblob

In [ ]:
from textblob import TextBlob

In [ ]:
score_list=[]

for text in data['Reviews']:
  
  sentence = TextBlob(text)
  
  score = sentence.sentiment.polarity
  score_list.append(score)

In [ ]:
import statistics
median = statistics.median(score_list)
print(median)

In [ ]:
sentiment_label = []
for score in score_list:
  if score > median:
    sentiment_label.append('positive')
  elif score < median:
    sentiment_label.append('negative')
  else:
    sentiment_label.append('neutral')
print(sentiment_label)

In [ ]:
dataset = data
dataset['SCORE']=score_list
dataset['SENTIMENT_LABEL']=sentiment_label
print(dataset)
dataset.to_csv("TextBlobResultswithMedian.csv")
!cp TextBlobResultswithMedian.csv "Results"